<a href="https://colab.research.google.com/github/pakyeon/deep-rl-class-ko/blob/main/notebooks/ko/bonus-unit1/bonus-unit1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 보너스 유닛 1: 강아지 허기를 막대기 물어오게 훈련시키자 🐶

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit2/thumbnail.png" alt="보너스 유닛 1 썸네일">

이 노트북에서는 첫 번째 유닛에서 배운 내용을 강화하면서 **허기 강아지가 막대기를 물어오도록 훈련시키고, 이후 브라우저에서 직접 놀 수 있도록 해볼 거예요.**

⬇️ 아래는 **이 유닛을 마치면 달성할 수 있는 예시**입니다. ⬇️ (▶을 눌러 실행해 보세요)

In [ ]:
%%html
<video controls autoplay><source src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy.mp4" type="video/mp4"></video>

### 환경 🎮

- Huggy the Dog은 [Puppo The Corgi](https://blog.unity.com/technology/puppo-the-corgi-cuteness-overload-with-the-unity-ml-agents-toolkit)를 기반으로 [Thomas Simonini](https://twitter.com/ThomasSimonini)가 만든 환경입니다.

### 사용된 라이브러리 📚

- [MLAgents](https://github.com/Unity-Technologies/ml-agents)

우리는 튜토리얼을 지속적으로 개선하고 있으니, **이 노트북에서 문제가 발견되면** [GitHub 저장소에 이슈를 등록](https://github.com/huggingface/deep-rl-class/issues)해 주세요.

## 이 노트북의 목표 🏆

이 노트북을 마치면 다음을 할 수 있게 됩니다:

- **허기를 훈련시키기 위해 사용된 상태 공간, 행동 공간, 보상 함수**를 이해합니다.  
- **직접 허기를 훈련시켜** 막대기를 물어오게 합니다.  
- **훈련시킨 허기와 브라우저에서 직접 놀 수 있게** 됩니다.

## 이 노트북은 딥 강화학습 강의의 일부입니다  
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="딥 강화학습 강의 일러스트"/>

이 무료 강의에서는 다음을 배우게 됩니다:

- 📖 **이론과 실습**을 통해 딥 강화학습을 학습합니다.  
- 🧑‍💻 Stable Baselines3, RL Baselines3 Zoo, CleanRL, Sample Factory 2.0 같은 **유명한 딥 RL 라이브러리 사용법**을 익힙니다.  
- 🤖 **독특한 환경에서 에이전트를 훈련**합니다.  

더 자세한 내용은 📚 [강의 계획서](https://simoninithomas.github.io/deep-rl-course)를 확인하세요 👉

**<a href="http://eepurl.com/ic5ZUD">강의에 가입하는 것</a>도 잊지 마세요!**  
(이메일을 수집하는 이유는 각 유닛이 공개될 때 링크를 보내드리고, 도전 과제 및 업데이트 정보를 전달하기 위함입니다.)

커뮤니티 및 강사들과 소통하고 싶다면 디스코드 서버에 참여하세요 👉🏻 https://discord.gg/ydHrjt3WP5

## 사전 준비 사항 🏗️

이 노트북을 시작하기 전에 다음을 완료해야 합니다:

🔲 📚 **강화학습의 기초 개념**(몽테카를로, 시간차 학습, 보상 가설 등)을 이해하려면 유닛 1을 학습하세요.

🔲 📚 **허기에 대한 소개**를 이해하려면 보너스 유닛 1을 완료하세요.

## GPU 설정하기 💪  
- **에이전트의 훈련 속도를 높이기 위해 GPU를 사용**합니다. 이를 위해 `런타임 > 런타임 유형 변경`으로 이동하세요.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU 설정 1단계">

- `Hardware Accelerator > GPU`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Step 2">

## 저장소 클론하기 🔽

- **ML-Agents**가 포함된 저장소를 클론해야 합니다.

In [2]:
%%capture
# 저장소 클론 (약 3분 소요)
!git clone --depth 1 https://github.com/Unity-Technologies/ml-agents

## 가상 환경 설정하기 🔽

- **ML-Agents**가 Colab에서 정상적으로 실행되려면, Colab의 Python 버전이 해당 라이브러리의 요구사항을 충족해야 합니다.

- 지원되는 Python 버전은 `setup.py` 파일의 `python_requires` 항목에서 확인할 수 있습니다. 이 파일들은 **ML-Agents** 라이브러리를 설치할 때 필요하며, 다음 경로에 위치합니다:
  - `/content/ml-agents/ml-agents/setup.py`
  - `/content/ml-agents/ml-agents-envs/setup.py`

- 현재 Colab의 Python 버전(`!python --version`으로 확인 가능)은 이 요구사항과 일치하지 않으며, 이로 인해 설치가 조용히 실패하고 나중에 다음과 같은 오류가 발생할 수 있습니다:
  - `/bin/bash: line 1: mlagents-learn: command not found`
  - `/bin/bash: line 1: mlagents-push-to-hf: command not found`

- 이를 해결하기 위해, **ML-Agents** 라이브러리와 호환되는 Python 버전으로 가상 환경을 생성하겠습니다.

`참고:` *향후 호환성을 위해, 항상 설치 파일의 `python_requires` 값을 확인하고, Colab의 Python 버전이 호환되지 않을 경우 아래 스크립트에서 설정된 최대 지원 버전으로 가상 환경을 설정하세요.*

In [ ]:
# Colab의 현재 Python 버전(ML-Agents와 호환되지 않음)
!python --version

Python 3.11.11


In [ ]:
# virtualenv 설치 및 가상 환경 생성
!pip install virtualenv
!virtualenv myenv

# Miniconda 다운로드 및 설치
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# Miniconda 활성화 후 Python 3.10.12 설치
!source /usr/local/bin/activate
!conda install -q -y --prefix /usr/local python=3.10.12 ujson  # 여기서 Python 버전을 지정

# Python 및 conda 경로에 대한 환경 변수 설정
!export PYTHONPATH=/usr/local/lib/python3.10/site-packages/
!export CONDA_PREFIX=/usr/local/envs/myenv

In [1]:
# 새로운 가상 환경의 Python 버전(ML-Agents와 호환)
!python --version

Python 3.10.12


## 종속성 설치 🔽

In [ ]:
%%capture
# 저장소 안으로 들어가서 패키지를 설치하세요(3분 정도 소요)
%cd ml-agents
!pip3 install -e ./ml-agents-envs
!pip3 install -e ./ml-agents

## 환경 zip 파일을 다운로드하고 `./trained-envs-executables/linux/`로 이동하기

- 환경 실행 파일은 zip 파일 안에 있습니다.
- 이를 다운로드하여 `./trained-envs-executables/linux/` 위치에 배치해야 합니다.

In [ ]:
!mkdir ./trained-envs-executables
!mkdir ./trained-envs-executables/linux

`wget`를 사용하여 https://github.com/huggingface/Huggy에서 Huggy.zip 파일을 다운로드했습니다.

In [ ]:
!wget "https://github.com/huggingface/Huggy/raw/main/Huggy.zip" -O ./trained-envs-executables/linux/Huggy.zip

In [ ]:
%%capture
!unzip -d ./trained-envs-executables/linux/ ./trained-envs-executables/linux/Huggy.zip

파일에 접근할 수 있는지 확인하세요.

In [ ]:
!chmod -R 755 ./trained-envs-executables/linux/Huggy

## 이 환경이 어떻게 작동하는지 복습해 봅시다

### 상태 공간: Huggy가 "인식하는" 것

Huggy는 환경을 "보지" 않습니다. 대신, 우리는 그에게 환경에 대한 정보를 제공합니다:

- 목표물(막대기) 위치
- 자신과 목표물 사이의 상대적 위치
- 자신의 다리 방향

이 모든 정보를 바탕으로 Huggy는 **목표를 달성하기 위해 다음에 취할 행동을 결정할 수 있습니다**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy.jpg" alt="Huggy" width="100%">

### 행동 공간: Huggy가 할 수 있는 움직임
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy-action.jpg" alt="Huggy action" width="100%">

**관절 모터가 Huggy의 다리를 구동합니다**. 이는 목표물에 도달하기 위해 Huggy가 **움직일 수 있도록 각 다리의 관절 모터를 올바르게 회전시키는 법을 배워야 한다**는 의미입니다.

### 보상 함수

보상 함수는 **Huggy가 목표를 달성하도록** 설계되었습니다: 막대기를 가져오는 것입니다.

강화 학습의 기초 중 하나는 *보상 가설*입니다: 목표는 **예상 누적 보상의 최대화**로 표현될 수 있습니다.

여기서 우리의 목표는 Huggy가 **너무 많이 회전하지 않으면서 막대기를 향해 가는 것**입니다. 따라서 보상 함수는 이 목표를 반영해야 합니다.

우리의 보상 함수:

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/reward.jpg" alt="Huggy reward function" width="100%">

- *방향 보너스*: **목표물에 가까워지는 것에 대해 보상**합니다.
- *시간 패널티*: 모든 행동마다 주어지는 고정 시간 패널티로, **가능한 빨리 막대기에 도달하도록** 합니다.
- *회전 패널티*: Huggy가 **너무 많이 회전하거나 너무 빠르게 회전하면** 패널티를 줍니다.
- *목표 도달 보상*: Huggy가 **목표물에 도달하면** 보상합니다.

## Huggy 설정 파일 생성하기

- ML-Agents에서는 **config.yaml 파일에 학습 하이퍼파라미터를 정의합니다.**

- 이 노트북의 범위에서는 하이퍼파라미터를 수정하지 않을 것입니다. 하지만 실험으로 시도해보고 싶다면 다른 하이퍼파라미터도 수정해 보세요. Unity는 [각 파라미터를 설명한 좋은 문서](https://github.com/Unity-Technologies/ml-agents/blob/main/docs/Training-Configuration-File.md)를 제공합니다.

- Huggy를 위한 설정 파일을 만들어야 합니다.

  - 화면 왼쪽에 있는 폴더 아이콘을 클릭하세요.

  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit1/create_file.png" alt="Create file" width="10%">

  - `/content/ml-agents/config/ppo`로 이동하세요
  - 마우스 우클릭하고 `Huggy.yaml`이라는 새 파일을 생성하세요

  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit1/create-huggy.png" alt="Create huggy.yaml" width="20%">

- 아래 내용을 복사하여 붙여넣으세요 🔽

In [ ]:
behaviors:
  Huggy:
    trainer_type: ppo
    hyperparameters:
      batch_size: 2048
      buffer_size: 20480
      learning_rate: 0.0003
      beta: 0.005
      epsilon: 0.2
      lambd: 0.95
      num_epoch: 3
      learning_rate_schedule: linear
    network_settings:
      normalize: true
      hidden_units: 512
      num_layers: 3
      vis_encode_type: simple
    reward_signals:
      extrinsic:
        gamma: 0.995
        strength: 1.0
    checkpoint_interval: 200000
    keep_checkpoints: 15
    max_steps: 2e6
    time_horizon: 1000
    summary_freq: 50000

- 파일을 저장하는 것을 잊지 마세요!

- **하이퍼파라미터를 수정하고 싶은 경우**, Google Colab 노트북에서 다음 위치에서 config.yaml을 열 수 있습니다: `/content/ml-agents/config/ppo/Huggy.yaml`

- 예를 들어 **학습 중에 더 많은 모델을 저장하고 싶다면** (현재는 200,000 학습 타임스텝마다 저장), 다음을 수정해야 합니다:
  - `checkpoint_interval`: 각 체크포인트 사이에 수집되는 학습 타임스텝 수
  - `keep_checkpoints`: 유지할 모델 체크포인트의 최대 수

=> `checkpoint_interval`을 줄이면 **Hub에 업로드할 모델이 더 많아지고 업로드 시간이 길어진다**는 점을 명심하세요.
이제 에이전트를 학습할 준비가 되었습니다 🔥.

## 에이전트 학습하기

에이전트를 학습하려면 **mlagents-learn을 실행하고 환경이 포함된 실행 파일을 선택**하면 됩니다.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/mllearn.png" alt="ml learn function" width="100%">

ML Agents에서는 학습 스크립트를 실행합니다. 네 가지 매개변수를 정의합니다:

1. `mlagents-learn <config>`: 하이퍼파라미터 설정 파일 경로
2. `--env`: 환경 실행 파일 위치
3. `--run-id`: 학습 실행에 부여할 이름
4. `--no-graphics`: 학습 중 시각화를 실행하지 않음

모델을 학습하고 중단 시 학습을 계속하려면 `--resume` 플래그를 사용하세요.

> `--resume`을 처음 사용할 때는 오류가 발생합니다. 오류를 우회하려면 블록을 다시 실행하세요.

학습은 기기에 따라 30~45분 정도 소요됩니다(반드시 **GPU를 설정**하세요). 커피 한 잔 마시러 가세요 ☕️, 당신은 그럴 자격이 있습니다 🤗.

In [ ]:
!mlagents-learn ./config/ppo/Huggy.yaml --env=./trained-envs-executables/linux/Huggy/Huggy --run-id="Huggy2" --no-graphics

## 에이전트를 🤗 Hub에 업로드하기

- 에이전트 학습이 완료되었으므로 **이제 브라우저에서 Huggy를 가지고 놀 수 있도록 Hub에 업로드할 준비가 되었습니다🔥.**

커뮤니티에 모델을 공유하기 위한 세 가지 추가 단계입니다:

1️⃣ (아직 완료하지 않았다면) HF 계정 생성 ➡ https://huggingface.co/join

2️⃣ 로그인 후, Hugging Face 웹사이트에서 인증 토큰을 저장해야 합니다.
- 새 토큰 생성(https://huggingface.co/settings/tokens) **쓰기 권한으로**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="HF 토큰 생성">

- 토큰 복사
- 아래 셀을 실행하고 토큰 붙여넣기

In [2]:
from huggingface_hub import notebook_login
notebook_login()

Google Colab이나 Jupyter Notebook을 사용하지 않는 경우, 대신 다음 명령어를 사용하세요: `huggingface-cli login`

그런 다음, 간단히 `mlagents-push-to-hf`를 실행하면 됩니다.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/mlpush.png" alt="ml learn 함수" width="100%">

4가지 매개변수를 정의합니다:

1. `--run-id`: 학습 실행 ID의 이름.
2. `--local-dir`: 에이전트가 저장된 위치, results/<run_id 이름>이므로 제 경우에는 results/First Training.
3. `--repo-id`: 생성하거나 업데이트하려는 Hugging Face 저장소 이름. 항상 <huggingface 사용자 이름>/<저장소 이름> 형식
저장소가 없으면 **자동으로 생성됩니다**
4. `--commit-message`: HF 저장소는 git 저장소이므로 커밋 메시지를 정의해야 합니다.

In [ ]:
!mlagents-push-to-hf --run-id="HuggyTraining" --local-dir="./results/Huggy2" --repo-id="ThomasSimonini/ppo-Huggy" --commit-message="Huggy"

모든 과정이 정상적으로 완료되면 (다른 URL과 함께) 프로세스 끝에 다음과 같은 내용이 표시됩니다:

```
Your model is pushed to the hub. You can view your model here: https://huggingface.co/ThomasSimonini/ppo-Huggy
```

이것은 모델 저장소 링크입니다. 저장소에는 모델 사용 방법을 설명하는 모델 카드, Tensorboard 로그, 구성 파일이 포함되어 있습니다. **이 저장소가 git 저장소라는 점이 특히 훌륭합니다. 다양한 커밋을 생성하고, 새로운 푸시로 저장소를 업데이트하고, Pull Request를 열 수 있습니다.**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/modelcard.png" alt="ml learn 함수" width="100%">

하지만 이제 최고의 부분입니다: **온라인에서 Huggy와 놀 수 있습니다 👀.**

## 허기와 놀아보기 🐕

이 단계는 가장 간단합니다:

- 브라우저에서 Huggy 게임을 엽니다: https://huggingface.co/spaces/ThomasSimonini/Huggy  
- **Play with my Huggy model** 버튼을 클릭하세요.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/load-huggy.jpg" alt="허기 불러오기" width="100%">

1. 1단계에서는 **모델 저장소**를 선택하세요. 이는 모델 ID입니다 (예: `ThomasSimonini/ppo-Huggy`).

2. 2단계에서는 **재생할 모델을 선택**하세요:
   - 저는 훈련 중 500,000 스텝마다 모델을 저장했기 때문에 여러 개가 있습니다.
   - 가장 최신 모델을 사용하고 싶어서 `Huggy.onnx`를 선택했습니다.

👉 **에이전트의 성능 향상을 확인하려면 다양한 시점의 모델로 시도해보는 것이 좋습니다.**

보너스 유닛을 완료한 걸 축하합니다! 🎉

이제 편히 앉아 허기 🐶와 함께 노는 시간을 즐겨보세요.  
그리고 **허기를 친구들과 공유하며 즐거움을 나누는 것도 잊지 마세요 🤗**  
소셜 미디어에 공유할 때는 **@huggingface 와 @simoninithomas 를 태그**해 주세요!

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy-cover.jpeg" alt="허기 커버" width="100%">

## 계속 배워나가고, 멋진 하루 보내세요 🤗